In [26]:
import numpy as np
from scipy.integrate import odeint
from scipy.integrate import simps
import pandas as pd

In [27]:
M = 1000000
a_1 = 10.
b_1 = 1.

mu     = np.random.gamma(a_1, b_1, M)
beta   = np.random.gamma(a_1, b_1, M)
nu     = np.random.gamma(a_1, b_1, M)
sigma  = np.random.gamma(a_1, b_1, M)
kappa  = np.random.gamma(a_1, b_1, M)
K      = np.random.gamma(a_1, b_1, M)

mu_hat     = np.random.gamma(a_1, b_1, M)
beta_hat   = np.random.gamma(a_1, b_1, M)
nu_hat     = np.random.gamma(a_1, b_1, M)
sigma_hat  = np.random.gamma(a_1, b_1, M)
kappa_hat  = np.random.gamma(a_1, b_1, M)
K_hat      = np.random.gamma(a_1, b_1, M)

# Mínimos y máximos
mu_m,mu_M       = 1e-5,0.1
beta_m,beta_M   = 1e-5,0.99
nu_m,nu_M       = 1e-5,0.1
sigma_m,sigma_M = 1e-4,0.9
kappa_m,kappa_M = 1e-6,1e-4
K_m,K_M         = 1e-2,4.

# Normalización de parámetros
mu = (mu_M - mu_m)* ( mu - min(mu) ) / ( max(mu) - min(mu) ) + mu_m
beta = (beta_M - beta_m)* ( beta - min(beta) ) / ( max(beta) - min(beta) ) + beta_m
nu = (nu_M - nu_m)* ( nu - min(nu) ) / ( max(nu) - min(nu) ) + nu_m
sigma = (sigma_M - sigma_m)* ( sigma - min(sigma) ) / ( max(sigma) - min(sigma) ) + sigma_m
kappa = (kappa_M - kappa_m)* ( kappa - min(kappa) ) / ( max(kappa) - min(kappa) ) + kappa_m
K = (K_M - K_m)* ( K - min(K) ) / ( max(K) - min(K) ) + K_m

mu_hat = (mu_M - mu_m)* ( mu_hat - min(mu_hat) ) / ( max(mu_hat) - min(mu_hat) ) + mu_m
beta_hat = (beta_M - beta_m)* ( beta_hat - min(beta_hat) ) / ( max(beta_hat) - min(beta_hat) ) + beta_m
nu_hat = (nu_M - nu_m)* ( nu_hat - min(nu_hat) ) / ( max(nu_hat) - min(nu_hat) ) + nu_m
sigma_hat = (sigma_M - sigma_m)* ( sigma_hat - min(sigma_hat) ) / ( max(sigma_hat) - min(sigma_hat) ) + sigma_m
kappa_hat = (kappa_M - kappa_m)* ( kappa_hat - min(kappa_hat) ) / ( max(kappa_hat) - min(kappa_hat) ) + kappa_m
K_hat = (K_M - K_m)* ( K_hat - min(K_hat) ) / ( max(K_hat) - min(K_hat) ) + K_m

In [28]:
def model (x,t,p):
    """
    p[0]=mu,p[1]=beta,p[2]=nu,p[3]=sigma,p[4]=kappa, p[5]=K
    """
    fx = np.zeros(3)
    xi = 1e-5
    fx[0] = p[0] - ((p[1]*x[1]) + p[2]  + xi  )*x[0]
    fx[1] = ((p[1]*x[1])+ xi ) *x[0] - (p[3] + p[2])*x[1]
    fx[2] = p[3]*x[1] - (p[4] + p[2])*x[2]
    return fx

In [29]:
# En construcción
def R0 (p):
    x0 = np.array([1,0.00000001,0.00000194])
    time = np.linspace(0.0,34.0,35)
    nn = len(time)
    dt = 1.0/(10.0*nn)
    n_quad = 10.0*nn+1
    t_quad = np.linspace(time[0],time[-1],n_quad)
    x = odeint(model,x0,t_quad,args=(p,))
    S = x[:,0]
    I = x[:,1]
    beta = p[1]
    sigma = p[2]
    nu = p[3]
    f = beta / (sigma + nu)
    phi = K * simps(f) * dt
    return phi
    

In [30]:
def prevalenceHIV (p):
    x0 = np.array([1,0.00000001,0.00000194])
    time = np.linspace(0.0,34.0,35)
    nn = len(time)
    dt = 1.0/(10.0*nn)
    n_quad = 10*nn+1
    t_quad = np.linspace(time[0],time[-1],n_quad)
    x = odeint(model,x0,t_quad,args=(p,))
    S = x[:,0]
    I = x[:,1]
    beta = p[1]
    K = p[5]
    xi = 1e-5
    f = (beta * I + xi) * S
    phi = K * simps(f) * dt
    return phi

In [31]:
def prevalenceAIDS (p):
    x0 = np.array([1,0.00000001,0.00000194])
    time = np.linspace(0.0,34.0,35)
    nn = len(time)
    dt = 1.0/(10.0*nn)
    n_quad = 10*nn+1
    t_quad = np.linspace(time[0],time[-1],n_quad)
    x = odeint(model,x0,t_quad,args=(p,))
    I = x[:,1]
    sigma = p[3]
    K = p[5]
    xi = 1e-5
    f = sigma * I
    phi = K * simps(f) * dt
    return phi

In [32]:
p=[1e-5, 1e-5, 1e-5, 1e-4, 1e-6,1e-2]
y_X = prevalenceHIV(p)
print(y_X)

9.999997790349065e-08


In [33]:
# Algorithm to compute Sensitivity Indices

A = np.column_stack((mu,beta, nu, sigma,kappa,K))
B = np.column_stack((mu_hat,beta_hat, nu_hat, sigma_hat,kappa_hat,K_hat))

numcols = len(A[0])
C_list = list()
i=0
while i < numcols:
    B_copy = np.copy(B)
    B_copy[:,i] = A[:,i]
    C_list.append(B_copy)
    i+=1

print("Dimensión de A: " + str(np.shape(A)))
print("Dimensión de B: " + str(np.shape(B)))
print("Dimensión de C: " + str(np.shape(C_list)))

np.savetxt('A.txt', A)
np.savetxt('B.txt', B)

Dimensión de A: (1000000, 6)
Dimensión de B: (1000000, 6)
Dimensión de C: (6, 1000000, 6)


In [34]:
y_A = np.zeros((M,1))
y_B = np.zeros((M,1))
y_C = np.zeros((M,len(C_list)))

for j in range(M):
    y_A[j] = prevalenceHIV(A[j,:])
    y_B[j] = prevalenceHIV(B[j,:])

for i in range(len(C_list)):
    for j in range(M):
        y_C[j][i]= prevalenceHIV(C_list[i][j,:])
        
print(np.shape(y_A))
print(np.shape(y_B))
print(np.shape(y_C))
print(y_A)
print(y_B)
print(y_C)

(1000000, 1)
(1000000, 1)
(1000000, 6)
[[3.02388704e-05]
 [1.50575188e-05]
 [8.89045420e-06]
 ...
 [4.00825533e-05]
 [2.90334158e-05]
 [2.23626944e-05]]
[[1.85692018e-05]
 [3.79328035e-05]
 [1.91184783e-04]
 ...
 [9.92671595e-05]
 [3.21217996e-05]
 [5.37018832e-05]]
[[3.40904763e-05 1.72012457e-05 1.58306811e-05 2.40589588e-05
  1.85692018e-05 1.68332082e-05]
 [1.77578119e-04 1.34704223e-05 1.94025255e-05 8.81259257e-05
  3.79328035e-05 2.87933642e-05]
 [1.08460043e-04 9.23589932e-06 4.37307829e-03 4.37448486e-04
  1.91184783e-04 1.11336729e-04]
 ...
 [6.30659826e-05 5.37394466e-03 4.84534861e-05 8.71981855e-05
  9.92671595e-05 2.82568286e-05]
 [5.26086009e-05 1.92585687e-05 6.76592273e-05 1.32744275e-05
  3.21217996e-05 5.12265985e-05]
 [2.78233898e-05 4.25024025e-05 8.69760631e-05 2.87233190e-05
  5.37018832e-05 5.98256884e-05]]


In [35]:
y_A_mean = np.sum(y_A)/M
y_B_mean = np.sum(y_B)/M
f_0_sqr = y_A_mean * y_B_mean

print(y_A_mean)
print(y_B_mean)
print(f_0_sqr)

0.0015506248932101062
0.0017155059170150526
2.6601061793727714e-06


In [36]:
S = []

sum_yA_sqr = 0
for i in range(M):
    sum_yA_sqr += y_A[i] * y_A[i]
var_Y = (1/M)* sum_yA_sqr - f_0_sqr
    
for i in range(len(C_list)):
    sum_yA_yC = 0    
    for j in range(M):
        sum_yA_yC += y_A[j] * y_C[j][i]       
    var_e_y_q_i = sum_yA_yC/M - f_0_sqr
    print(var_e_y_q_i)
    S.append(var_e_y_q_i/var_Y)

print("sum_yA_sqr: " + str(sum_yA_sqr))
print("var_Y: " + str(var_Y))
print(S)

[1.95288635e-06]
[9.24269855e-06]
[1.4110512e-06]
[2.81668174e-06]
[5.20138193e-08]
[8.04531003e-07]
sum_yA_sqr: [47.60289107]
var_Y: [4.49427849e-05]
[array([0.04345272]), array([0.20565478]), array([0.03139661]), array([0.06267261]), array([0.00115733]), array([0.01790123])]


In [37]:
ST = []

for i in range(len(C_list)):
    sum_yB_yC = 0
    for j in range(M):
        sum_yB_yC += y_B[j] * y_C[j][i]
    var_e_y_q_i = (1/M)* sum_yB_yC - f_0_sqr
    ST.append(1- (var_e_y_q_i/var_Y))
    
print(ST)

[array([0.35584337]), array([0.81329908]), array([0.26704823]), array([0.09432593]), array([-0.08259376]), array([-0.08474156])]


In [38]:
indices = np.row_stack((np.transpose(S),np.transpose(ST)))
sobol_indices = pd.DataFrame(indices, index = (['Si', 'STi']), columns= ['mu', 'beta', 'nu', 'sigma', 'kappa', 'K'])
print(sobol_indices)

           mu      beta        nu     sigma     kappa         K
Si   0.043453  0.205655  0.031397  0.062673  0.001157  0.017901
STi  0.355843  0.813299  0.267048  0.094326 -0.082594 -0.084742
